In [1]:
import django 
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()
from sch.models import *
import datetime as dt
from django.db.models import CharField
from sch.actions import *
from sch.xviews import week

In [18]:
slot = s.slots.all()[200]
slot.fillableBy()

<EmployeeManager [<Employee: Mona>, <Employee: Danica>, <Employee: Josh>, <Employee: Tiffany>]>
<SlotManager [{'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}, {'employee': None}]>


<EmployeeManager []>

In [ ]:
Schedule.objects.all

In [ ]:
from sch.actions import *
from sch.xviews import week

In [ ]:
week.get_days("2022-W42")

<WorkdayManager []>

In [ ]:
emps = Employee.objects.in_other_slot(workday=Workday.objects.get(slug='2022-10-31'),shift=Shift.objects.get(name__contains='7C'))
emps = emps.filter(shifts_trained=Shift.objects.get(name__contains='7C'))
emps

DoesNotExist: Workday matching query does not exist.

In [ ]:
shiftsPerWeek = 0
for sft in Shift.objects.filter(start__hour__gte=10, cls='CPhT').values_list('occur_days',flat=True):
    shiftsPerWeek += len(sft)
shiftsPerWeek

28

## NIGHT PERCENTAGE OF TOTAL SHIFTS"

In [ ]:
NIGHT_PERCENTS = []
for emp in Employee.objects.all():
    if Slot.objects.filter(employee=emp).exists():
        perc = round(Slot.objects.filter(employee=emp,shift__start__hour__gte=10).count()/Slot.objects.filter(employee=emp).count()*100, 1)
        NIGHT_PERCENTS += [f"{emp}: {perc}% EVENINGS  {int(perc/4) *'+'}"]
        
NIGHT_PERCENTS
        

[]

## SLOTS FROM A WEEK

In [ ]:
def getTradableSlots(slot):
    print('INPUT: ', slot)
    emp = slot.employee
    sft = slot.shift
    wd = slot.workday
    weekdays = slot.workday.siblings_iweek
    daysOn = Slot.objects.filter(workday__in = weekdays,employee = emp).values('workday').distinct()
    daysOff = weekdays.exclude(pk__in=daysOn)
    print('daysOff:',list(daysOff.values_list('date__day',flat=True)))
    tenable = []
    slotDaysOff = Slot.objects.filter(workday__in=daysOff, shift__in=emp.shifts_available.all()).order_by('workday')
    print(slotDaysOff)
    for slotB in slotDaysOff:
        if emp in slot.fillableBy():
            if slotB.employee in slot.fillableBy():
                tenable.append(slotB)
    print('tenableTrades:',len(tenable),[f'{i.shift}>{i.workday.date.day}' for i in tenable])
    return tenable



In [ ]:
slots = Slot.objects.filter(employee__name="Trisha",workday__iweek=43,workday__date__year=2022)
for slot in slots:
    print('-------------')
    print(slot.is_turnaround)
    getTradableSlots(slot)

In [ ]:
wds = Workday.objects.filter(date__year=2022,ischedule=7)
ss = Slot.objects.filter(workday__in=wds)
ss

<SlotManager [<Slot: 221016-tMI None>, <Slot: 221016-pI None>, <Slot: 221016-t7C None>, <Slot: 221016-t7P None>, <Slot: 221016-pC None>, <Slot: 221016-tEI None>, <Slot: 221016-tEP None>, <Slot: 221016-p3 None>, <Slot: 221016-tN None>, <Slot: 221016-pN None>, <Slot: 221016-pS None>, <Slot: 221016-pM None>, <Slot: 221016-t3 None>, <Slot: 221017-tMI None>, <Slot: 221017-tS None>, <Slot: 221017-pI None>, <Slot: 221017-t7C None>, <Slot: 221017-t7P None>, <Slot: 221017-pPC None>, <Slot: 221017-tOP None>, '...(remaining elements truncated)...']>

In [ ]:
for wd in Workday.objects.filter(iweek=44):
    print(wd,"~~~Week:",wd.week)

2022 10 30 ~~~Week: WEEK-2022.44
2022 10 31 ~~~Week: WEEK-2022.44
2022 11 01 ~~~Week: WEEK-2022.44
2022 11 02 ~~~Week: WEEK-2022.44
2022 11 03 ~~~Week: WEEK-2022.44
2022 11 04 ~~~Week: WEEK-2022.44
2022 11 05 ~~~Week: WEEK-2022.44


In [ ]:
n = Workday.objects.all().count()
print( f"TOTAL WORKDAYS: {n} " )

TOTAL WORKDAYS: 88 


In [ ]:
from django import forms
import datetime as dt
class ArticleForm(forms.Form):
     title = forms.CharField()
     pub_date = forms.DateField()

In [ ]:
""" 
Janet’s ducks lay 16 eggs per day. She eats three for breakfast 
every morning and bakes muffins for her friends every day with 
four. She sells the remainder for $2 per egg. How much does she
make every day? SHOW YOUR CALCULATIONS
"""

# Define the number of eggs Janet's ducks lay per day.
eggs_per_day = 16

# Define the number of eggs Janet eats for breakfast every morning.
eggs_for_breakfast = 3

# Define the number of eggs Janet uses to bake muffins for her friends every day.
eggs_for_muffins = 4

# Define the price Janet sells her eggs for.
eggs_price = 2

# Calculate the number of eggs Janet has left over after eating breakfast and baking muffins.
eggs_leftover = eggs_per_day - eggs_for_breakfast - eggs_for_muffins

# Calculate the amount of money Janet makes selling her eggs.
eggs_money = eggs_leftover * eggs_price

# Print the number of eggs Janet has left over after eating breakfast and baking muffins.
print("Janet has " + str(eggs_leftover) + " eggs left over after eating breakfast and baking muffins.")

# Print the amount of money Janet makes selling her eggs.
print("Janet makes $" + str(eggs_money) + " selling her eggs.")

Janet has 9 eggs left over after eating breakfast and baking muffins.
Janet makes $18 selling her eggs.


In [ ]:
def turnarounds_withTrades ():
    tradeable = {}
    tas = Slot.objects.filter(is_turnaround=True)|Slot.objects.filter(is_preturnaround=True)
    slts = tas.filter(employee__evening_pref=True)
    empl_wantsThisSwapped = slts.filter(shift__start__hour__lt=10)
    for i in range(empl_wantsThisSwapped.count()):
        s = empl_wantsThisSwapped[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('pref'):
                tradeable['pref'] = []
            tradeable['pref'] += [s]
    empl_likesThisSlot = slts.filter(shift__start__hour__gte=10)
    for i in range(empl_likesThisSlot.count()):
        s = empl_likesThisSlot[i]
        if s.fillableBy().count() > 0:
            if not tradeable.get('nonpref'):
                tradeable['nonpref'] = []
            tradeable['nonpref'] += [s]
    return tradeable
    
x = turnarounds_withTrades()
x

{}

In [ ]:
t = {}
a = t.get('a')
print(a)

None


In [ ]:
def weekSlots_for_Empl (yearN, weekN, employee):
    slots = Slot.objects.filter(workday__iweek=weekN,workday__date__year=yearN,employee=employee)
    for i in range(7):
        s = slots.filter(workday__iweekday=i)
        if s.exists() == False:
            yield None
        else:
            yield s.first()
            
def display_weekSlots_for_Empl (yearN, weekN, employee):
    slots = [slot for slot in weekSlots_for_Empl(yearN,weekN,employee)]
    wds = 'SUN MON TUE WED THU FRI SAT'.split(' ')
    empl = f'{employee}'
    
    print(f'{yearN}-Week_{weekN} employee schedule:')
    print(f'_________ {empl} __________')
    for i in range(7):
        print(f'{wds[i]}: {slots[i]}')
    print('==========================')

In [ ]:
slots = weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))
slots = [slot for slot in slots]
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Molly'))
display_weekSlots_for_Empl(2022,44,Employee.objects.get(name='Josh'))

2022-Week_44 employee schedule:
_________ Molly __________
SUN: None
MON: None
TUE: None
WED: None
THU: None
FRI: None
SAT: None
2022-Week_44 employee schedule:
_________ Josh __________
SUN: None
MON: None
TUE: None
WED: None
THU: None
FRI: None
SAT: None


In [ ]:
def emp1_emp2_cowork_days (emp1,emp2,yearN,schN):
    jtsSlots = Slot.objects.filter(employee=emp1,workday__date__year=yearN,workday__ischedule=schN)
    nkiSlots = Slot.objects.filter(employee=emp2,workday__date__year=yearN,workday__ischedule=schN)
    dates = jtsSlots.union(nkiSlots)
    dates = dates.values('workday')
    for d in dates:
        if d in jtsSlots.values('workday'):
            if d in nkiSlots.values('workday'):
                yield d['workday']
                # "[207,217,223...]"
josh = Employee.objects.get(name="Josh")    
nicki = Employee.objects.get(name="Nicki")      
xs = emp1_emp2_cowork_days(josh,nicki,2022,5)
days = Workday.objects.filter(pk__in=xs)


print(f'COWORK DAYS FOR {josh} and {nicki}')
print("TOTAL FOR SCHEDULE:", days.count())
print(list(days.values_list('slug',flat=True)))



COWORK DAYS FOR Josh and Nicki
TOTAL FOR SCHEDULE: 0
[]


In [ ]:
josh_slots_w_nicki = Slot.objects.filter(workday__in=days,employee=josh)
josh_slots_w_nicki 
for jslot in josh_slots_w_nicki:
    nslot = Slot.objects.get(workday=jslot.workday,employee=nicki)
    print(jslot,"  |  ",nslot)

## Employee Training Comparisons

In [ ]:
vals = Employee.objects.annotate(n_shifts_trained=Count('shifts_trained')).order_by('n_shifts_trained').values_list('name','n_shifts_trained')
for val in vals:
    print ("%s is trained for %s shift(s)." % (val[0], val[1]))

Amanda is trained for 1 shift(s).
Bara is trained for 1 shift(s).
Brianna S is trained for 1 shift(s).
Brittanie is trained for 1 shift(s).
Corwin is trained for 1 shift(s).
Mark B is trained for 1 shift(s).
Mary is trained for 1 shift(s).
Michael is trained for 1 shift(s).
Dianne is trained for 2 shift(s).
Mei is trained for 2 shift(s).
Erin is trained for 3 shift(s).
Kylee is trained for 3 shift(s).
Mona is trained for 3 shift(s).
Tony is trained for 3 shift(s).
Brianna is trained for 4 shift(s).
Cheryl is trained for 4 shift(s).
David is trained for 4 shift(s).
Elisia is trained for 4 shift(s).
Nicki is trained for 4 shift(s).
Jaden is trained for 5 shift(s).
Leslie is trained for 5 shift(s).
Kat is trained for 6 shift(s).
Esperanza is trained for 7 shift(s).
Kristen is trained for 7 shift(s).
Sam is trained for 7 shift(s).
Tiffany is trained for 7 shift(s).
Alex is trained for 8 shift(s).
Andrew is trained for 8 shift(s).
Carol is trained for 8 shift(s).
Carrie is trained for 8 shi

In [ ]:
Schedule.objects.create()

(4, {'sch.Workday': 1, 'sch.Week': 1, 'sch.Period': 1, 'sch.Schedule': 1})

In [ ]:
for emp in Employee.objects.all():
    print(f"{emp.name.ljust(20)} {emp.yrs_experience} yr. experience")

Josh                 4.63 yr. experience
Sabrina              4.63 yr. experience
Brianna              4.63 yr. experience
Molly                5.24 yr. experience
Doug                 15.75 yr. experience
Brittanie            1.91 yr. experience
Esperanza            1.07 yr. experience
Carrie               4.63 yr. experience
Elisia               0.15 yr. experience
Jaden                0.32 yr. experience
Lynlee               5.24 yr. experience
Carol                4.63 yr. experience
Trisha               4.63 yr. experience
Danica               1.88 yr. experience
Tiffany              1.07 yr. experience
Brianna S            4.63 yr. experience
Amanda               5.46 yr. experience
Michael              4.63 yr. experience
Mona                 4.63 yr. experience
Mary                 4.63 yr. experience
Mark B               4.63 yr. experience
Andrew               4.63 yr. experience
Mei                  4.63 yr. experience
Tony                 13.39 yr. experience
Sam           

In [ ]:
b = Employee.objects.get(name="David")
b.hire_date = dt.date(2022,6,1)
b.save()
b.hire_date

datetime.date(2022, 6, 1)

# Low Option Employees

In [ ]:
loe = Employee.objects.filter(fte__gt=0).filter(n_trained__lt=4)
for l in loe:
    print (l.name, l.n_trained, l.shifts_trained.all())

Brittanie 1 <ShiftManager [<Shift: tOP>]>
Brianna S 1 <ShiftManager [<Shift: pMO>]>
Amanda 1 <ShiftManager [<Shift: tN>]>
Michael 1 <ShiftManager [<Shift: tN>]>
Mona 3 <ShiftManager [<Shift: tEI>, <Shift: tEP>, <Shift: t3>]>
Mary 1 <ShiftManager [<Shift: pN>]>
Mark B 1 <ShiftManager [<Shift: pN>]>
Mei 2 <ShiftManager [<Shift: p3>, <Shift: pN>]>
Tony 3 <ShiftManager [<Shift: pE>, <Shift: p3>, <Shift: pN>]>
Corwin 1 <ShiftManager [<Shift: pS>]>


In [ ]:
import itertools as itr 

iters = list(itr.permutations(josh.shifts_available.all(), r=2))
iters

[(<Shift: tMI>, <Shift: tS>),
 (<Shift: tMI>, <Shift: t7C>),
 (<Shift: tMI>, <Shift: t7P>),
 (<Shift: tMI>, <Shift: tOP>),
 (<Shift: tMI>, <Shift: tEI>),
 (<Shift: tMI>, <Shift: tEP>),
 (<Shift: tMI>, <Shift: tN>),
 (<Shift: tMI>, <Shift: t3>),
 (<Shift: tS>, <Shift: tMI>),
 (<Shift: tS>, <Shift: t7C>),
 (<Shift: tS>, <Shift: t7P>),
 (<Shift: tS>, <Shift: tOP>),
 (<Shift: tS>, <Shift: tEI>),
 (<Shift: tS>, <Shift: tEP>),
 (<Shift: tS>, <Shift: tN>),
 (<Shift: tS>, <Shift: t3>),
 (<Shift: t7C>, <Shift: tMI>),
 (<Shift: t7C>, <Shift: tS>),
 (<Shift: t7C>, <Shift: t7P>),
 (<Shift: t7C>, <Shift: tOP>),
 (<Shift: t7C>, <Shift: tEI>),
 (<Shift: t7C>, <Shift: tEP>),
 (<Shift: t7C>, <Shift: tN>),
 (<Shift: t7C>, <Shift: t3>),
 (<Shift: t7P>, <Shift: tMI>),
 (<Shift: t7P>, <Shift: tS>),
 (<Shift: t7P>, <Shift: t7C>),
 (<Shift: t7P>, <Shift: tOP>),
 (<Shift: t7P>, <Shift: tEI>),
 (<Shift: t7P>, <Shift: tEP>),
 (<Shift: t7P>, <Shift: tN>),
 (<Shift: t7P>, <Shift: t3>),
 (<Shift: tOP>, <Shift: tMI